Review : word2vec
랜덤 단어 벡터들로 시작하여 전체 corpus의 각 단어마다 반복. 워드 벡터들을 이용하여 주변의 단어들을 예측하는 모델(context word vector와 center word vector간의 내적을 활용) 학습 시 vector들을 update하여 주변 단어를 더욱 잘 예측하도로 설계. 이후 softmax 함수를 사용하여 probability 도출. 이는 Bag of Words 모델이라 불림. 하지만, 단어의 순서를 고려하지 않는다는 단점. 또한, 이전에 없던 새로운 단어가 들어왔을 때 확률을 제대로 예측하기 어렵다는 단점. word2vec은 학습과정에서 고차원의 공간 내에 비슷한 단어끼리 가까이 배치함.
3. Optimization : Gradient Descent
그렇다면 word vector들을 어떻게 잘 학습시킬 수 있을까?
word vector들을 random하게 0과 가까운 작은 수로 초기화한다. 손실함수는 J로 파라미터인 세타를 변경하여 최대의 J의 값을 같는 것을 목표로 학습을 진행한다. 이 알고리즘의 idea는 현재 세타의 값에서 손실함수 J의 기울기를 계산하고 경사하강법을 진행하는 것이다.
하지만, 손실함수 J는 corpus에는 매우 많은 단어가 포함되어 있기에 이를 반복하여 기울기나 세타를 미분하는 것은 굉장히 많은 비용이 소요됨. 따라서, 신경망 알고리즘에서는 경사하강법을 사용하지 않음. 대신 이를 변형하여 하나의 center word나 32개의 center words들과 같은 작은 배치를 토대로 경사하강법을 진행하는 Stochastic Gradient Descent(SGD)를 사용함. 물론 전체 corpus에 대해 계산한 것이 아니기에 정확하지는 않지만 컴퓨팅 비용 줄여줌. 또한, 모델이 복잡할수록 SGD가 좋은 효율을 가짐. 하지만 역시 학습에 사용되는 데이터의 양이 극히 적기 때문에 희박한 경사하강법이 될 수 밖에 없음. 또한, 실제 등장하는 단어 벡터에 대해서만 update를 진행하고자 할 때도 있을 것이다.
보통 사용되는 딥러닝에서 word vector는 행 벡터로 제시됨. word2vec을 사용할 때에 한 단어당 두 개의 word vector를 사용한다고 했었는데, 쉬운 optimization을 위해 그렇게 사용하며 마지막에는 이를 평균을 내서 사용한다. 물론, 한 벡터만 사용할 수도 있지만, 이는 오히려 알고리즘을 더욱 복잡하게 만든다.
Word2Vec이 제시되었을 때, 두 개의 모델이 존재했다. 하나는 Skip-gram(SG) : center word를 토대로 context words를 예측하는 모델/ 나머지는 Continuous Bag of Words(CBOW) : context words를 토대로 center word를 예측함. 또한 효율적인 학습을 위해 실제 논문에서는 Naive softmax보다 negative sampling을 진행함. Naive softmax를 진행할 때, 분모에서 전체 vocabulary에 대해 내적을 실행해야 하기 때문에 비용이 많이 듬. 따라서, negative sampling에서는 True pair(center word와 context window)와 몇몇 noise pairs(center word와 random word)를 비교하는 binary logistic regression을 학습하고자 한다. objective function에서는 logistic(sigmoid) function을 사용하여 결과값을 0과 1사이로 matching하도록 하는데, true pair에는 단지 sigmoid함수 뒤에 log만 취하는 반면, noise pair에는 sigmoid함수와 log 안에 -1을 곱해버림. 이로 인해, 목적함수가 center word와 context word간의 유사도는 높게 가져가면서, center word와 random word간의 유사도는 낮게 가져가도록 만들 수 있음. 랜덤 단어를 선택할 때, unigram distribution of words를 사용함. 만약 10억개의 word corpus 내에 특정 단어가 90번 등장했다면, 90/10억으로 그 단어의 unigram probability를 계산할 수 있다. 이후 해당 단어를 3/4승 하여 단어 간의 격차를 줄여주고 이후 전체 u(w)3/4승의 합인 Z로 나누어주어 각 단어의 추출 확률을 설정한다.
Co-occurence Matrix : 특정 단어와 다른 단어 간의 등장 빈도에 대한 matrix. window와 symmetric or non-symmetric을 설정. window를 paragraph or full document로 설정할 수도 있음. 하지만 co-occurence Matrix는 많은 양의 storage가 필요한만큼 고차원이며, sparse하며 randomness를 많이 가질 수 있다. 이를 해결하고자 각 단어의 중요한 정보만 고정된, 작은 수의 차원의로 저장할 수 있는데 이를 dense vector라고 한다.(보통 약 25~1000개 차원) 그렇다면 어떻게 dense vector를 만들 수 있나???
SVD(Singular Value Decomposition)을 통해 가능 - 이를 활용하여 truncated SVD를 통해 차원 축소 가능. 하지만 SVD를 통해 word vector를 만든다면 성능이 좋지 못함. 따라서, 빈도에 log를 취한다거나 단어의 최소 임계값을 정하여 전처리를 수행한다던가, function words(a,the)를 고려하지 않는 방안으로 개선할 수 있다. 실제로 이를 활용한 COALS model이 있음. 
count-based model(LSA, COAKS,,,)은 빠른 학습시간과 효율적인 통계정보를 사용하지만, 단어의 유사성 여부만을 파악하는데 사용되며 단어 간 관계는 파악이 불가할 뿐만 아니라, 큰 빈도수에 과도한 중요도를 부여한다는 단점을 갖고 있었다. direct prediction model(word2vec, NNLM) 등은 높은 수준의 성능을 보이고 단어 유사성 이상의 복잡한 패턴을 파악하지만, 말뭉치 크기가 성능에 영향을 미치며 효율적으로 통계정보를 사용하지 못한다는 단점을 갖고 있다. word vector를 만드는 또다른 알고리즘인 GLOVE는 count-based model과 direct prediction을 융합하려했다. glove는 vector의 차이에서 의미 요소를 반영하려 했다. co-occurence 확률이 meaning components를 반영할 수 있다고 생각한 것이다. co-occurence 확률을 어떻게 선형적인 구성요소로 표현하여 벡터를 더하고 뺌에 따라 의미가 함축될 수 있을까? log-bilinear model(내적 대신 두 단어 간 동시 등장할 조건부확률에 log를 취한 것이 두 단어 벡터의 내적과 같다!!) with vector differences(a가 존재할 때 x의 등장확률을 b가 존재할때 x의 등장확률로 나눈 후 로그를 취한 것이 a,b 벡터 간 차와 x벡터의 내적과 같다!!). GLOVE model에서는 두 단어의 내적이 co-occurence probability와 비슷하게 되도록 목적함수를 설정하려 했다. 또한, f function을 사용하여 단어의 빈도수가 많음에 따라 중요도가 높아지는 현상을 제한하려고 했다. 또한, 최적화과정에서는 co-occurence matrix를 직접적으로 수정함에 따라 학습속도 또한 빠르게 유지하였다. 
* word2vec 알고리즘은 center word만 보고 negative sampling을 생성한 후에 예측함. whole co-occurence matrix를 보고 최적화를 한다면, entire vector를 보고 최적화할 수 있기에 word2vec은 statistical information은 적다.
NLP에서는 Intrinsic or Extrinsic evaluation이 존재한다. Intrinsic은 평가를 위한 데이터에 적용하여 성능을 평가하는 것으로 계산이 빠르며 시스템을 이해하기 쉽지만, 현실에서 해당 시스템이 유용한지 알 수 없다. Extrinsic Evaluation은 실제 task에 대한 성능 평가로 평가하는 데에 시간이 더욱 걸리고 문제점을 파악하기 어렵지만 때론 더 유용하다. Intrinsic Evaluation은 word analogy를 평가하는 문제를 들 수 있다. GLOVE 모델이 analogy evaluation에서 좋은 평가를 받았지만 교수님은 단지 더 좋은 data로 인해 그러한 결과가 나온 것일 수 있다고 말함. 효율적인 차원은 300차원 이하로 밝혀짐. extrinsic word vector evaluation에서도 좋은 평가. 

7. word senses : 하나의 단어는 오래될수록, 많은 의미를 담고 있음. 어떻게 하나의 word vector로 단어의 의미를 모두 나타낼 수 있지? 비슷한 단어끼리 클러스터링을 수행하는데 이때, bank1, bank2와 같이 다른 뜻을 가졌다면 다른 단어로 취급하여 클러스터링 수행. 하지만 실제로는 복잡하고 단어 간 구분이 불분명할 수 있기 때문에 수행하지 않음. 